# 서울시 자전거 대여건수

## pyspark 기본 셋팅

In [1]:
import os
import sys
import pyspark

myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

## DataFrame 생성

In [2]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

myBikeSchema = StructType([
    StructField("DATE", StringType(), True),
    StructField("COUNT", LongType(), True)
])

df = spark.read.format('csv')\
    .option("header",'True')\
    .option("encoding", "utf-8")\
    .schema(myBikeSchema)\
    .load('data/seoul_bike.csv')
df.show()

+----------+-----+
|      DATE|COUNT|
+----------+-----+
|2018-01-01| 4950|
|2018-01-02| 7136|
|2018-01-03| 7156|
|2018-01-04| 7102|
|2018-01-05| 7705|
|2018-01-06| 5681|
|2018-01-07| 5220|
|2018-01-08| 6309|
|2018-01-09| 5988|
|2018-01-10| 4476|
|2018-01-11| 4337|
|2018-01-12| 4401|
|2018-01-13| 3756|
|2018-01-14| 4675|
|2018-01-15| 6993|
|2018-01-16| 7421|
|2018-01-17| 6990|
|2018-01-18| 7054|
|2018-01-19| 8329|
|2018-01-20| 6148|
+----------+-----+
only showing top 20 rows



### 문제 1-1: 년도별 대여건수 합계

In [3]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

year_udf = udf(lambda x: x[10:12],StringType())
year_df = df.withColumn("year",year_udf(df.DATE))
year_df.groupBy('year').agg({"count":"sum"}).show()

+----+----------+
|year|sum(count)|
+----+----------+
|2019|   1871935|
|2018|  10124874|
+----+----------+



### 문제 1-2: 년도별, 월별 대여건수 합계

In [4]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

year_udf = udf(lambda x: x[:4],StringType())
month_udf = udf(lambda x: x[5:7],StringType())
df = df.withColumn("year",year_udf(df.DATE)).withColumn("month",month_udf(df.DATE))
df.groupBy('year').pivot('month').agg({"count":"sum"}).show()

+----+------+------+------+------+------+-------+-------+-------+-------+-------+------+------+
|year|    01|    02|    03|    04|    05|     06|     07|     08|     09|     10|    11|    12|
+----+------+------+------+------+------+-------+-------+-------+-------+-------+------+------+
|2019|495573|471543|904819|  null|  null|   null|   null|   null|   null|   null|  null|  null|
|2018|164367|168741|462661|687885|965609|1207123|1100015|1037505|1447993|1420621|961532|500822|
+----+------+------+------+------+------+-------+-------+-------+-------+-------+------+------+



### 문제 1-3: 년도별, 월별 대여건수 그래프